In [1]:
#Importamos pandas
import pandas as pd
import numpy as np
import ast
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk
from wordcloud import WordCloud
import re

# Análisis Exploratorio de Datos

Cargamos el archivo con los datos

In [2]:
# Carga tus datos
df = pd.read_csv('movies.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

Observamos las primeras filas de los datos

In [ ]:
df.head()

## Analisis de las variables

## Analisis General

In [47]:
x=df.info()
print(x) # Vemos un resumen de todas las variables disponibles

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45452 entries, 0 to 45451
Data columns (total 30 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   budget                      45452 non-null  float64
 1   id_movie                    45452 non-null  int64  
 2   original_language           45452 non-null  object 
 3   overview                    45313 non-null  object 
 4   popularity                  45452 non-null  float64
 5   release_date                45452 non-null  object 
 6   revenue                     45452 non-null  float64
 7   runtime                     45452 non-null  float64
 8   tagline                     45452 non-null  object 
 9   title                       45452 non-null  object 
 10  vote_average                45452 non-null  float64
 11  vote_count                  45452 non-null  float64
 12  release_year                45452 non-null  int64  
 13  return                      454

Vamos a iniciar descartando variables de nulo interes para el modelo como lo son: "collection_poster_path", "collection_backdrop_path", "spoken_languages_iso" y "status"

In [ ]:
df.drop(columns= ['collection_poster_path', "collection_backdrop_path", "spoken_languages_iso", "status"], axis= 1, inplace= True)

### Variables numericas

In [ ]:
# Reemplazar los valores 'inf' por 0 en la columna 'return'
df['return'] = df['return'].replace([float('inf')], 0)

df.describe() # Vamos a ver caracteristicas como la cantidad, la media y la dispersion de las variables numericas

- budget: Esta es la cantidad de dinero presupuestada para la película. El presupuesto promedio es de alrededor de 4.2 millones de dólares, aunque la desviación estándar es alta, lo que sugiere que mucha variacion entre los presupuestos de las peliculas en principio.

- id_movie: Este es un identificador único para cada película. No tiene sentido estadístico real, pero es útil para la indexación y el cruce con otros datos.

- popularity: El promedio de popularidad de las películas es de alrededor de 2.93, con una desviación estándar de 6, lo que sugiere una amplia gama de valores de popularidad.

- release_date y release_year: Fecha y año de lanzamiento de la película. Las películas en el conjunto de datos se lanzaron entre 1874 y 2020, con una tendencia hacia las películas más recientes.

- revenue: La recaudación promedio de las películas es de alrededor de 11.2 millones de dólares, pero con una gran desviación estándar que indica una amplia gama de valores de ingresos.

- runtime: El tiempo de duración promedio de las películas es de alrededor de 93.67 minutos, con una desviación estándar de 38.85, lo que indica una variación moderada en los tiempos de duración de las películas.

- vote_average: El puntaje promedio de la película es de alrededor de 5.62 con una desviación estándar de 1.92, lo que sugiere una variación moderada en los puntajes de las películas.

- vote_count: El conteo de votos varía ampliamente, con una media de alrededor de 109.98 y una desviación estándar de 491.35.

- return: representa la relación entre los ingresos y el presupuesto de una película. Un retorno más alto indica que una película generó más ingresos en relación con su presupuesto. Sin embargo, la falta de datos completos puede hacer que algunos valores de retorno sean engañosos. Por ejemplo, una película con un presupuesto muy pequeño y ganancias moderadas puede tener un retorno muy alto, mientras que una película con un gran presupuesto y grandes ganancias puede tener un retorno más bajo. Aun así, el retorno puede ser una variable útil para entender qué tan exitosa fue una película financieramente.

### Correlacion entre variables

In [ ]:
correlation_matrix = df.corr()

Elaboramos un heat map para poder vizualizar las correlaciones de mejor forma

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.show()

Analisis del Heat Map

Del Heat Map se desprenden lo siguiente:

- budget y revenue: Con una correlación de 0.768665, estas dos características parecen tener una fuerte correlación positiva. Esto tiene sentido porque a menudo cuanto más grande es el presupuesto de una película, más alto es el ingreso esperado.

- popularity y vote_count: Con una correlación de 0.559847, estas dos características también tienen una fuerte correlación positiva. Esto también es intuitivo porque las películas más populares tienden a tener más votos.

- vote_count y revenue: Con una correlación de 0.811994, estas dos características también tienen una fuerte correlación positiva. Esto sugiere que las películas con más votos tienden a generar más ingresos, lo que tiene sentido ya que las películas más populares a menudo son las que generan más ingresos.

- return parece no estar fuertemente correlacionado con ninguna de las otras características, lo que podría indicar que la rentabilidad de una película (es decir, su ingreso dividido por el presupuesto) no se ve fuertemente afectada por ninguna de las otras características en el conjunto de datos.

## Analisis de "budget"

Vamos a analizar por separado estas variables, iniciamos por budget, que representa el presupuesta de la pelicula. Empecemos viendo la distribucion de esta variable.

In [ ]:
# Utilizar la función value_counts para obtener la distribución de frecuencia de la variable 'budget'
distribucion_frecuencia = df['budget'].value_counts()

# Crear el gráfico de densidad
plt.figure(figsize=(10, 6))  # Tamaño del gráfico
sns.kdeplot(data=df, x='budget', fill=True, color='skyblue', alpha=0.7)  # Gráfico de densidad
plt.xlabel('Budget')  # Etiqueta del eje x
plt.ylabel('Density')  # Etiqueta del eje y
plt.title('Density Plot of Budget')  # Título del gráfico
plt.grid(True)  # Agregar líneas de cuadrícula
plt.show()  # Mostrar el gráfico

# Mostrar el resultado
print(distribucion_frecuencia)

In [ ]:
# Calcular el total de películas en el DataFrame
total_peliculas = len(df)

# Filtrar las películas con un presupuesto de 0
peliculas_sin_presupuesto = df[df['budget'] == 0]

# Calcular el total de películas sin presupuesto
total_peliculas_sin_presupuesto = len(peliculas_sin_presupuesto)

# Calcular el porcentaje
porcentaje_peliculas_sin_presupuesto = (total_peliculas_sin_presupuesto / total_peliculas) * 100

# Mostrar el resultado
print(f"El {porcentaje_peliculas_sin_presupuesto:.2f}% de las películas del DataFrame no tienen presupuesto cargado.")

Vemos que de el total de peliculas el 80,4% no tiene ningun presupuesto cargado en la base de datos, este numero es muy grande y hace poco util calcular la media o la desviacion estandar con los datos como estan ya que los resultados no seran representativos.

## Analisis de "revenue"

In [ ]:
# Utilizar la función value_counts para obtener la distribución de frecuencia de la variable 'budget'
distribucion_frecuencia = df['revenue'].value_counts()

# Crear el gráfico de densidad
plt.figure(figsize=(10, 6))  # Tamaño del gráfico
sns.kdeplot(data=df, x='revenue', fill=True, color='skyblue', alpha=0.7)  # Gráfico de densidad
plt.xlabel('Revenue')  # Etiqueta del eje x
plt.ylabel('Density')  # Etiqueta del eje y
plt.title('Density Plot of Revenue')  # Título del gráfico
plt.grid(True)  # Agregar líneas de cuadrícula
plt.show()  # Mostrar el gráfico

# Mostrar el resultado
print(distribucion_frecuencia)

In [ ]:
# Calcular el total de películas en el DataFrame
total_peliculas = len(df)

# Filtrar las películas con un presupuesto de 0
peliculas_sin_presupuesto = df[df['revenue'] == 0]

# Calcular el total de películas sin presupuesto
total_peliculas_sin_presupuesto = len(peliculas_sin_presupuesto)

# Calcular el porcentaje
porcentaje_peliculas_sin_presupuesto = (total_peliculas_sin_presupuesto / total_peliculas) * 100

# Mostrar el resultado
print(f"El {porcentaje_peliculas_sin_presupuesto:.2f}% de las películas del DataFrame no tienen cargado sus ingresos.")

Al igual que con budget, revenue no tiene datos en el 83,66% del dataframe, lo que hace que no sea una buena variable decisora, intentar completar los datos con la mediana no seria util en este caso por la gran cantidad de datos faltantes que existen, la variable no va a aportar gran valor a nuestro modelo

## Variable "return"

In [ ]:
# Utilizar la función value_counts para obtener la distribución de frecuencia de la variable 'budget'
distribucion_frecuencia = df['return'].value_counts()

# Crear el gráfico de densidad
plt.figure(figsize=(10, 6))  # Tamaño del gráfico
sns.kdeplot(data=df, x='return', fill=True, color='skyblue', alpha=0.7)  # Gráfico de densidad
plt.xlabel('Return')  # Etiqueta del eje x
plt.ylabel('Density')  # Etiqueta del eje y
plt.title('Density Plot of Return')  # Título del gráfico
plt.grid(True)  # Agregar líneas de cuadrícula
plt.show()  # Mostrar el gráfico

# Mostrar el resultado
print(distribucion_frecuencia)

In [ ]:
# Calcular el total de películas en el DataFrame
total_peliculas = len(df)

# Filtrar las películas con un presupuesto de 0
peliculas_sin_presupuesto = df[df['return'] == 0]

# Calcular el total de películas sin presupuesto
total_peliculas_sin_presupuesto = len(peliculas_sin_presupuesto)

# Calcular el porcentaje
porcentaje_peliculas_sin_presupuesto = (total_peliculas_sin_presupuesto / total_peliculas) * 100

# Mostrar el resultado
print(f"El {porcentaje_peliculas_sin_presupuesto:.2f}% de las películas del DataFrame no tienen cargado informacion de su retorno.")

Resultado logico ya que returno es budget/revenue, esta variable aportara muy poco valor a nuestro modelo.

## Variable popularity

In [ ]:
# Utilizar la función value_counts para obtener la distribución de frecuencia de la variable 'budget'
distribucion_frecuencia = df['popularity'].value_counts()

# Gráfico de densidad para la distribución de "popularity"
plt.figure(figsize=(8, 6))
sns.histplot(data=df, x='popularity', bins=30, kde=True, color='skyblue', edgecolor='k')
plt.xlabel('Popularity')
plt.ylabel('Density')
plt.title('Density Plot of Popularity')
plt.grid(True)
plt.show()

# Mostrar el resultado
print(distribucion_frecuencia)

Por el histograma obtenido parece que la variable popularity tiene outliers de pelicular con un valor de popularidad muy grandes, vamos a verlo en un boxplot

In [ ]:
# Crear el boxplot para la distribución de "popularity"
plt.figure(figsize=(10, 4))
sns.boxplot(data=df, x='popularity', color='skyblue')
plt.ylabel('Popularity')
plt.title('Boxplot of Popularity')
plt.grid(True)
plt.show()

Se pueden apreciar por la forma achatada del grafico de caja en la izquierda que la variable tiene muchos outliers que pueden estar afectando el analisis al momento de usar esta variable.

En primer lugar para saber como tratar con estos datos vamos a intentar identificarlos, para eso vamos a utilizar la tecnica estadística de los límites de Tukey para detectar outliers. Este método considera outliers aquellos valores que están por encima del tercer cuartil más 1.5 veces el rango intercuartil (IQR) o por debajo del primer cuartil menos 1.5 veces el IQR son outliers.

In [ ]:
# Método de Tukey para identificar outliers
Q1 = df['popularity'].quantile(0.25)
Q3 = df['popularity'].quantile(0.75)
IQR = Q3 - Q1
umbral_superior = Q3 + 1.5 * IQR
umbral_inferior = Q1 - 1.5 * IQR

# Identificar los outliers
outliers = df[(df['popularity'] < umbral_inferior) | (df['popularity'] > umbral_superior)]

In [ ]:
outliers.sort_values(by= 'popularity', ascending= False)

Vemos que hay 4211 registros que se categorizan como outliers, vamosa  comparar estos valores atipicos con los que no son atipicos mediante la feature "vote_count" que habiamos viste que estaban correlacionadas positivamente.

In [ ]:
# Máscara booleana para los registros que no son outliers
mascara_no_outliers = ~df.index.isin(outliers.index)

# DataFrame sin outliers
df_no_outliers = df[mascara_no_outliers]

print(df_no_outliers['vote_average'].describe())

print(outliers['vote_average'].describe())

Vamos a realizar una prueba de hipotesis para determinar si el vote_average estadisticamente es igual para el conjunto sin outliers y el conjunto de outliers.

In [ ]:
from scipy import stats

# Realiza la prueba t de Student para muestras independientes.
t_statistic, p_value = stats.ttest_ind(df_no_outliers['vote_average'], outliers['vote_average'])

# Imprime los resultados.
print("Estadística t:", t_statistic)
print("Valor p:", p_value)

# Realiza la decisión basada en el valor p y un nivel de significancia (α) de 0.05.
alpha = 0.05
if p_value < alpha:
    print("Rechazamos la hipótesis nula. Las poblaciones tienen medias diferentes.")
else:
    print("No podemos rechazar la hipótesis nula. Las poblaciones podrían tener la misma media.")

Estadisticamente, con un nivel de confianza del 95%, los outliers si cuentan con una cantidad de votos mayor a los que no son outliers, ya que su media es mayor, lo que nos podria indicar que estos valores son correctos, no son valores incorrectos sino que realmente son peliculas muy populares.

Para cerrar el analisis de esta variable a continuacion vamos ver cuales son el top 10 de peliculas con mayor popularidad

In [ ]:
# Ordenar el DataFrame por popularidad en orden descendente
df_top_popularity = df.sort_values(by='popularity', ascending=False)

# Tomar las 10 primeras filas del DataFrame ordenado (Top 10)
df_top_10 = df_top_popularity.head(10)

# Ordenar el DataFrame por popularidad en orden descendente
df_top_popularity = df.sort_values(by='popularity', ascending=False)

# Tomar las 10 primeras filas del DataFrame ordenado (Top 10)
df_top_10 = df_top_popularity.head(10)

# Crear el gráfico de barras para el Top 10 de películas con mayor popularidad
plt.figure(figsize=(10, 6))
colors = sns.color_palette('Set3', n_colors=len(df_top_10))  # Paleta de colores
plt.bar(df_top_10['title'], df_top_10['popularity'], color=colors)
plt.xlabel('Película')
plt.ylabel('Popularidad')
plt.title('Top 10: Películas con Mayor Popularidad')
plt.xticks(rotation=45, ha='right')  # Rotar las etiquetas del eje x para mayor legibilidad
plt.grid(axis='y')  # Agregar rejillas en el eje y para guiar la lectura
plt.tight_layout()  # Ajustar los espacios para una mejor presentación
plt.show()

Con base en estos hallazgos, podemos concluir que la variable "popularity" es un indicador relevante para medir el interés y la relevancia de una película en la plataforma de TMDB. Las películas con una popularidad excepcionalmente alta pueden ser consideradas como aquellas que han capturado la atención y el interés del público, lo que podría ser un factor importante para tener en cuenta en el análisis y la toma de decisiones relacionadas con el contenido y la promoción.

### Variables categoricas

In [ ]:
# Convierte los strings a listas
df['genero_names'] = df['genero_names'].apply(ast.literal_eval)

# "Aplana" las listas, cuenta los valores, y selecciona los 10 primeros
top_10_generos = df['genero_names'].explode().value_counts()[:10]

print(top_10_generos)

In [ ]:
top_10_collections = df['name_collection'].explode().value_counts()[:10]
print(top_10_collections)

In [ ]:
df['production_companies_names'] = df['production_companies_names'].apply(ast.literal_eval)
top_10_productoras = df['production_companies_names'].explode().value_counts()[:10]
print(top_10_productoras)

In [ ]:
df['production_countries_names'] = df['production_countries_names'].apply(ast.literal_eval)
top_10_paises=df['production_countries_names'].explode().value_counts()[:10]
print(top_10_paises)

In [ ]:
top_10_or_languaje = df['original_language'].explode().value_counts()[:10]
print(top_10_or_languaje)

In [ ]:
fig, axs = plt.subplots(3, 2, figsize=(15, 15))

top_10_generos = top_10_generos.sort_values(ascending=False)
top_10_productoras = top_10_productoras.sort_values(ascending=False)
top_10_paises = top_10_paises.sort_values(ascending=False)
top_10_collections = top_10_collections.sort_values(ascending=False)
top_10_languages = top_10_or_languaje.sort_values(ascending=False)

axs[0, 0].barh(top_10_generos.index, top_10_generos.values, color='blue')
axs[0, 0].set_title('Top 10 Géneros más comunes')
axs[0, 0].invert_yaxis()

axs[0, 1].barh(top_10_productoras.index, top_10_productoras.values, color='orange')
axs[0, 1].set_title('Top 10 Productoras más comunes')
axs[0, 1].invert_yaxis()

axs[1, 0].barh(top_10_paises.index, top_10_paises.values, color='green')
axs[1, 0].set_title('Top 10 Países más comunes')
axs[1, 0].invert_yaxis()

# Solo si aún tienes suficientes datos para mostrar después de la filtración
if not top_10_collections.empty:
    axs[1, 1].barh(top_10_collections.index, top_10_collections.values, color='red')
    axs[1, 1].set_title('Top 10 Collections más comunes')
    axs[1, 1].invert_yaxis()

axs[2, 0].barh(top_10_or_languaje.index, top_10_or_languaje.values, color='purple')
axs[2, 0].set_title('Top 10 Idiomas Originales más comunes')
axs[2, 0].invert_yaxis()

# Eliminamos el último gráfico vacío si no es necesario
fig.delaxes(axs[2,1])

plt.tight_layout()
plt.show()

- Géneros de películas: Los géneros más comunes en las películas de esta base de datos son el Drama, la Comedia y el Thriller, con 20302, 13195 y 7635 películas, respectivamente. Estos son seguidos por Romance, Acción, Horror y Crimen.

- Colecciones de películas: La mayoría de las películas en la base de datos no pertenecen a ninguna colección, con 40955 películas. Entre las que sí pertenecen a colecciones, las más comunes son "The Bowery Boys", "Totò Collection", y las colecciones de "James Bond" y "Pokémon", cada una con 26-29 películas.

- Productoras de películas: Las compañías productoras más comunes en esta base de datos son Warner Bros. (1250 películas), Metro-Goldwyn-Mayer (MGM) (1080 películas) y Paramount Pictures (1007 películas). Estas son seguidas por Twentieth Century Fox Film Corporation, Universal Pictures, Columbia Pictures Corporation, entre otras.

- Países de producción de películas: Las producciones de películas son principalmente de los Estados Unidos, seguido por el Reino Unido, Francia, Alemania, e Italia.

- Idioma original de las películas: Las películas son en su mayoría en inglés, seguido por francés, italiano, japonés, y alemán.



In [ ]:
# Descomponer la columna 'genero_names'
data_exploded = df.explode('genero_names')

# Calcular la popularidad promedio y la calificación promedio de los votos por género
genero_popularity = data_exploded.groupby('genero_names')['popularity'].mean().sort_values(ascending=False)
genero_vote_average = data_exploded.groupby('genero_names')['vote_average'].mean().sort_values(ascending=False)

In [ ]:
print(genero_popularity)
print(genero_vote_average)

In [ ]:
# Crear un DataFrame con los datos agrupados
df_genre = pd.DataFrame({'Popularity': genero_popularity, 'Vote Average': genero_vote_average})

# Crear el gráfico de dispersión
plt.figure(figsize=(10,8))
plt.scatter(df_genre['Popularity'], df_genre['Vote Average'])

# Añadir etiquetas a los puntos
for i in range(df_genre.shape[0]):
    plt.text(df_genre['Popularity'][i], df_genre['Vote Average'][i], df_genre.index[i], 
             horizontalalignment='left', 
             verticalalignment='bottom', 
             fontsize=9)

# Añadir títulos y etiquetas a los ejes
plt.title('Popularidad y Calificación Promedio por Género')
plt.xlabel('Popularidad')
plt.ylabel('Calificación Promedio')

# Mostrar el gráfico
plt.show()


En el grafico arriba se observa que las peliculas mas populares y con mejor calificacion son las del genero "Animation" mientras que las menos populares son las "Foreing" y las "Documentary", que suelen ser para un tipo de audiencia mas espeficica.

In [ ]:
# Descomponer la columna 'production_countries_names'
data_exploded = df.explode('production_countries_names')

# Calcular la cantidad de películas, la popularidad promedio y la calificación de voto promedio por país
country_stats = data_exploded.groupby('production_countries_names').agg(
    cantidad_de_peliculas=('title', 'count'),
    popularity=('popularity', 'mean'),
    popularity_std=('popularity', 'std'),
    vote_average=('vote_average', 'mean'),
    vote_average_std=('vote_average', 'std')
)

# Ordenar los resultados por la cantidad de películas
country_stats = country_stats.sort_values(by='cantidad_de_peliculas', ascending=False)

country_stats.head(10)

- Producción de Películas: Estados Unidos de América es con creces el país que produce más películas, con un total de 21,176. Esto es más del doble que el siguiente país, Reino Unido, que ha producido 4,100 películas. La lista sigue con Francia, Alemania, Italia, Canadá, Japón, España, Rusia e India.

- Popularidad: A pesar de la gran cantidad de películas producidas por Estados Unidos, el Reino Unido tiene la popularidad media más alta, con un promedio de 4.331602, aunque la diferencia no es significativa respecto a la popularidad de las películas de Estados Unidos. Es importante destacar que la desviación estándar de la popularidad en el Reino Unido es mayor que en los Estados Unidos, lo que significa que hay más variabilidad en la popularidad de las películas británicas.

- Como ya habiamos visto en el Mapa de calor, no parece existir una relacion entre la popularidad de la pelicula y el promedio del puntaje.

- Calificación Promedio: A pesar de las diferencias en la cantidad de películas producidas y la popularidad, Japón lidera en términos de la calificación promedio más alta (6.163362), seguido de cerca por Francia y el Reino Unido. Las películas de Japón también tienen una desviación estándar relativamente alta para la calificación, lo que indica una mayor variabilidad en las calificaciones de las películas japonesas.

- Variabilidad: En términos de variabilidad (medida por la desviación estándar), las películas producidas en el Reino Unido tienen la mayor variabilidad tanto en popularidad como en calificación promedio. Esto podría indicar una gama más amplia de géneros y estilos en las películas del Reino Unido, o podría reflejar una mayor inconsistencia en la calidad de las películas.

- India y Rusia: India y Rusia, a pesar de tener una menor cantidad de películas producidas, tienen calificaciones promedio competitivas (5.945838 y 5.627552 respectivamente). Sin embargo, su popularidad media es significativamente menor que la de los otros países, lo que podría reflejar una menor distribución o visibilidad de sus películas a nivel internacional.

In [ ]:
# Director
directors = df.explode('director')
director_stats = directors.groupby('director').agg(
    cantidad_de_peliculas=('title', 'count'),
    popularity=('popularity', 'mean'),
    popularity_std=('popularity', 'std'),
    vote_average=('vote_average', 'mean'),
    vote_average_std=('vote_average', 'std')
)

# Actor principal
actor_stats = df.groupby('principal_actor').agg(
    cantidad_de_peliculas=('title', 'count'),
    popularity=('popularity', 'mean'),
    popularity_std=('popularity', 'std'),
    vote_average=('vote_average', 'mean'),
    vote_average_std=('vote_average', 'std')
)

In [ ]:
director_stats.sort_values(by='cantidad_de_peliculas', ascending=False).head(10)

Vamos a ver el comportamiento de las peliculas segun el director en funcion de la popularidad y la calificacion promedio

In [ ]:
# Obten los nombres de los cinco directores con más películas
top_directors = df['director'].value_counts().index[:5]

# Crea un nuevo DataFrame que contenga solo las películas de estos directores
top_director_movies = df[df['director'].isin(top_directors)]

# Crea un gráfico de dispersión con seaborn
plt.figure(figsize=(10,8))
sns.scatterplot(data=top_director_movies, x='popularity', y='vote_average', hue='director')
plt.title('Relación entre popularidad y puntuación media de voto para los cinco directores con más películas')

# Define los límites de los ejes
plt.xlim(0, 5)  # Aquí establecemos el rango de popularidad entre 0 y 50
plt.ylim(0, 10)  # Aquí establecemos el rango de vote_average entre 0 y 10

plt.show()

En el grafico anterior podemos ver como peliculas del director "Georges Melies" parecen tener un comportamiento similar en cuanto a las dos variables en juego, ademas, casi todas las peliculas se mantienen en un rango de voto promedio similar, pareciera que el director involucrado impacta en la critica del publico de forma relativamente similar en sus diferentes trabajos.

In [ ]:
actor_stats.sort_values(by='cantidad_de_peliculas', ascending=False).head(10)

Según los datos presentados, es evidente que tanto el director como el actor principal de una película podrían desempeñar un papel significativo en su puntuación y popularidad, siendo posiblemente factores valiosos para un modelo de recomendación.

Para los directores, figuras notables como Alfred Hitchcock y Woody Allen demuestran una alta consistencia en las puntuaciones promedio de sus películas, como se evidencia en la baja desviación estándar. Estos directores, que tienen una gran cantidad de películas en su haber, parecen tener una mayor probabilidad de recibir críticas positivas, lo que puede indicar que su estilo y habilidades se han perfeccionado con el tiempo.

Un patrón similar se observa entre los actores principales, con Clint Eastwood, Robert De Niro y Denzel Washington mostrando puntuaciones promedio altas y desviaciones estándar más bajas. Estos actores parecen tener una reputación de aparecer en películas bien recibidas, lo que puede aumentar la probabilidad de que las futuras películas en las que aparezcan también sean populares y bien calificadas.

In [ ]:
# Obten los nombres de los cinco actores con más películas
top_actors = df['principal_actor'].value_counts().index[:5]

# Crea un nuevo DataFrame que contenga solo las películas de estos actores
top_actors_movies = df[df['principal_actor'].isin(top_actors)]

# Crea un gráfico de dispersión con seaborn
plt.figure(figsize=(10,8))
sns.scatterplot(data=top_actors_movies, x='popularity', y='vote_average', hue='principal_actor')
plt.title('Relación entre popularidad y puntuación media de voto para los cinco actores con más películas')

# Define los límites de los ejes
plt.xlim(0, 10)  # Aquí establecemos el rango de popularidad entre 0 y 5
plt.ylim(0, 10)  # Aquí establecemos el rango de vote_average entre 0 y 10

plt.show()

En el caso de los actores principales se ve el grafico mucho mas disperso y no se puede identificar un patron o relacion entre el actor principal y la popularidad/promedio de voto.

### Analisis de outliers

Analisis de outliers de la feature popularity

In [ ]:
plt.figure(figsize=(12, 8))
sns.boxplot(x=df['popularity'])
plt.title('Boxplot de la popularidad de las películas')
plt.show()

In [ ]:
Q1 = df['popularity'].quantile(0.25)
Q3 = df['popularity'].quantile(0.75)
IQR = Q3 - Q1

# Definimos los límites para los outliers
lower_limit = Q1 - 1.5*IQR
upper_limit = Q3 + 1.5*IQR

# Filtramos los valores que son outliers
outliers = df[(df['popularity'] < lower_limit) | (df['popularity'] > upper_limit)]

# Imprimimos los outliers
outliers.sort_values(by='popularity',ascending= False).head()


Si vemos las peliculas que aparecen como Outliers vemos que son efectivamente peliculas muy populares, aunque son datos atipicos se ve razon para determinar que los mismos sean incorrectos

Primero, tomaremos las descripciones de las películas (o "resúmenes") y las procesaremos para prepararlas para el análisis. Este preprocesamiento puede incluir pasos como la eliminación de palabras irrelevantes (también conocidas como "stopwords"), la conversión de todas las palabras a minúsculas y la tokenización (la conversión de las descripciones de texto largo en listas de palabras individuales).

## Analisis de Columna Overview (Procesamiento de lenguaje natural)

Vamos a iniciar el trabajo limpiando un poco los datos

In [ ]:
# Los elementos sin descripcion se asignan como NaN para que no influyan en los resultado
df['overview'] = df['overview'].replace('No overview found.', np.nan)

Vamos a inicializar un lemmatizador y definir una funcion que hara el trabajo de normalizacion

In [ ]:
# Inicializa el lematizador
lemmatizer = WordNetLemmatizer()

def preprocess_text(text, print_progress=False):
    # Contador de iteraciones
    counter = 0

    # Eliminar signos de puntuación
    text = re.sub(r'[^\w\s]', '', text)

    # Cargar palabras vacías
    stop_words = stopwords.words('english')

    # Verifica si el texto es un número (incluyendo NaN)
    if isinstance(text, (int, float)):
        return ''

    # Convierte el texto a minúsculas
    text = text.lower()

    # Tokeniza el texto
    words = word_tokenize(text)

    # Elimina las palabras vacías
    words = [word for word in words if word not in stop_words]

    # Lematiza las palabras e imprime progreso
    new_words = []
    for word in words:
        new_word = lemmatizer.lemmatize(word)
        new_words.append(new_word)

        # Incrementa contador y verifica si imprimir progreso
        counter += 1
        if print_progress and counter % 1000 == 0:
            print(f"Processed {counter} words.")

    # Une las palabras en una cadena de texto
    text = " ".join(new_words)

    return text

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Aplica la función de preprocesamiento a cada descripción de película
df['overview'] = df['overview'].apply(preprocess_text)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter

# Crea una instancia de CountVectorizer
vectorizer = CountVectorizer()

# Ajustar y transformar los datos
X = vectorizer.fit_transform(df['overview'])

# Sumar las columnas de la matriz X para obtener el recuento total de cada palabra
word_counts = np.array(np.sum(X, axis=0)).flatten()

# Combinar los recuentos de palabras con los nombres de las palabras
words_counts_combined = zip(vectorizer.get_feature_names_out(), word_counts)

# Ordenar las palabras por recuento
words_counts_sorted = sorted(words_counts_combined, key=lambda x: x[1], reverse=True)

# Imprimir las 10 palabras más comunes
for word, count in words_counts_sorted[:10]:
    print(f'Word: {word}, Count: {count}')

In [ ]:
# Suma todas las veces que cada palabra aparece en nuestros textos
word_counts = np.asarray(X.sum(axis=0)).ravel()

# Asocia cada palabra con su recuento
word_counts_dict = dict(zip(vectorizer.get_feature_names_out(), word_counts))

# Genera la nube de palabras
wordcloud = WordCloud(width = 1000, height = 500, random_state=21, max_font_size=110).generate_from_frequencies(word_counts_dict)

plt.figure(figsize=(15, 8))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.show()


Las palabras más comunes en los resúmenes de las películas proporcionan una visión interesante de los temas recurrentes en las películas. "Life", "one" y "film" encabezan la lista, indicando que las tramas suelen girar en torno a historias de vida individuales y se presentan a menudo en un contexto cinematográfico o de filmación.

Palabras como "young", "man" y "love" son indicativos de tramas comunes en las películas que implican personajes jóvenes, a menudo hombres, en situaciones de amor o romance. La palabra "find" también es común, lo que sugiere una prevalencia de tramas basadas en la búsqueda o el descubrimiento.

"Year", "new" y "two" también aparecen con frecuencia, lo que podría indicar una tendencia a las tramas que ocurren durante un período de tiempo determinado (un año), que implican novedades o cambios, o que se centran en dos personajes o elementos principales.

En general, estos hallazgos sugieren que las tramas de las películas a menudo giran en torno a temas de amor, descubrimiento y cambio, y con frecuencia involucran a personajes jóvenes o a hombres en roles principales. Sin embargo, cabe destacar que estas son palabras bastante generales y, por lo tanto, es posible que no proporcionen una representación completa de la variedad de tramas y temas que se encuentran en las películas.

La nube de palabras nos muestra que palabras como "war", "father" y "murder" son bastante comunes en las descripciones de las películas en este dataset. Este hecho puede proporcionar cierta intuición sobre la temática de las películas.

In [ ]:
from sklearn.preprocessing import binarize

# Binarizar la matriz tfidf_matrix
tfidf_matrix_bin = binarize(X)

# Sumar los valores a lo largo del eje 0
word_appearances = np.asarray(tfidf_matrix_bin.sum(axis=0)).ravel()

# Asociar cada palabra con su recuento
word_appearances_dict = dict(zip(vectorizer.get_feature_names_out(), word_appearances))

# Ordenar el diccionario por los recuentos de apariciones de palabras de forma descendente
word_appearances_dict = dict(sorted(word_appearances_dict.items(), key=lambda item: item[1], reverse=True))

# Obten la cantidad total de películas
total_movies = X.shape[0]

# Calcular las proporciones y almacenarlas en un nuevo diccionario
word_appearances_proportions = {word: count / total_movies for word, count in word_appearances_dict.items()}

# Mostrar las 10 palabras más comunes y su proporción del total de películas
for word, proportion in list(word_appearances_proportions.items())[:20]:
    print(f"Word: {word}, Appears in: {proportion:.2%} of the movies")



Hay palabras que aparecen en una gran cantidad de peliculas como "life, one" que quizas no nos sirva para identificar la tematica de la pelicula, pero palabras mas especificas como "family, friend, war, love, young" que nos pueden ser utiles para identificar la tematica.

## Reducción de dimensionalidad y Modelado de Tópicos con LDA (Latent Dirichlet Allocation)

Utilizaremos el Algoritmo de Latent Dirichlet Allocation (LDA) para el modelado de tópicos. LDA asume que cada documento en un corpus está compuesto por una mezcla de tópicos y que cada palabra en el documento se puede atribuir a uno de los tópicos del documento. LDA utiliza un enfoque generativo, lo que significa que intenta volver a crear los documentos originales a partir de los tópicos que ha identificado.

Para utilizar LDA, primero preprocesaremos nuestros documentos (en este caso, descripciones de películas) realizando tareas como la eliminación de palabras vacías y la tokenización. Luego, crearemos una "bolsa de palabras" para cada documento, que es simplemente una lista de las palabras que aparecen en el documento junto con la frecuencia de cada palabra.

Luego, entrenaremos el modelo LDA en nuestras bolsas de palabras. El modelo aprenderá qué palabras tienden a aparecer juntas en los documentos y utilizará esta información para identificar los tópicos. Una vez entrenado, el modelo LDA podrá decirnos la proporción de cada tópico en cada documento, así como las palabras más representativas de cada tópico.

Finalmente, utilizaremos una métrica llamada "coherencia" para evaluar la calidad de nuestro modelo. La coherencia mide qué tan semánticamente similares son las palabras más representativas de cada tópico. Podemos utilizar la coherencia para comparar diferentes modelos o configuraciones de modelos, y para determinar el número óptimo de tópicos.

In [ ]:
import nltk
from nltk.tokenize import word_tokenize

# Descarga el tokenizador de NLTK si aún no lo has hecho
nltk.download('punkt')

# Asegurándome de que las descripciones sean strings
df['overview'] = df['overview'].apply(str)

# Creando una lista de todas las descripciones de las películas
movie_descriptions = df['overview'].tolist()

# Tokenizar las descripciones de las películas
movie_descriptions_tokenized = [word_tokenize(desc) for desc in movie_descriptions]

# Crear el diccionario
dictionary = corpora.Dictionary(movie_descriptions_tokenized)

# Crear el corpus
corpus = [dictionary.doc2bow(desc) for desc in movie_descriptions_tokenized]

for i in range(1,20):
    # Crear el modelo LDA
    lda = models.LdaModel(corpus, num_topics=i, id2word=dictionary, passes=2)

    # Calcular la métrica de coherencia
    coherence_model_lda = CoherenceModel(model=lda, texts=movie_descriptions_tokenized, dictionary=dictionary, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()

    print(f'Coherence Score:{i} es {coherence_lda}')


Basandonos en el parametro de coherencia vamos a seleccionar un numero de topicos de 14

In [ ]:
lda = models.LdaModel(corpus, num_topics=14, id2word=dictionary, passes=2)

In [ ]:
# Asignar cada película a su tema más probable
topics = [max(lda[corpus[i]], key=lambda x: x[1])[0] for i in range(len(movie_descriptions))]

# Contar cuántas películas pertenecen a cada tema
topic_counts = pd.Series(topics).value_counts()

print(topic_counts)

Estos números representan la cantidad de películas que el modelo LDA ha asignado a cada tópico. A partir de estos números, podemos obtener algunas ideas interesantes:

El tópico 13 es el más común, con 18961 películas asignadas. Esto puede sugerir que este tópico representa una temática muy general o popular en las películas de tu conjunto de datos.

Los tópicos 2, 9 y 6 también son relativamente comunes, cada uno con varios miles de películas asignadas.

En el otro extremo del espectro, los tópicos 1, 3 y 4 parecen ser menos comunes, con solo unas pocas docenas o cientos de películas asignadas a cada uno. Estos tópicos pueden representar temas más nicho o específicos que no se encuentran en una gran cantidad de películas.

In [ ]:
# Imprimir los 10 principales términos de cada tema
for idx, topic in lda.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic))
    print("\n")

## KeyWords de Overview

Vamos a obtener las palabras clave de cada resumen, de esta forma podremos comparar si entre los distintos resumenes existen coincidencias

In [34]:
movies = df

In [ ]:
def assign_topic_to_movies(lda_model, corpus, df):
    topic_assignments = []
    
    for i in range(len(corpus)):
        topics = lda_model.get_document_topics(corpus[i])
        topic_assignments.append(max(topics, key=lambda item: item[1])[0])
    
    df['topic'] = topic_assignments

# Llama a la función para asignar los tópicos a las películas
assign_topic_to_movies(lda, corpus, df)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

movies['overview'] = movies['overview'].apply(lambda x: '' if isinstance(x, float) else x)

# Esto es solo necesario la primera vez que uses NLTK
nltk.download('punkt')
nltk.download('stopwords')

# Inicializar el stemmer y las stopwords
stemmer = PorterStemmer()



In [ ]:
#Calcular TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(movies['processed_overview'])


In [ ]:
# Obtener las palabras clave para cada película
keywords = []
for i in range(tfidf_matrix.shape[0]):
    tfidf_scores = tfidf_matrix[i]
    words = vectorizer.inverse_transform(tfidf_scores)
    keywords.append(words)

movies['keywords'] = keywords

In [35]:
# Definir una función para limpiar y extraer las palabras clave
def extract_keywords(keywords_str):
    # Reemplaza los caracteres innecesarios
    keywords_str = keywords_str.replace("array([", "").replace("])", "")
    # Divide la cadena en palabras clave usando una coma como separador
    keywords_list = keywords_str.split(',')
    # Elimina los espacios adicionales y devuelve la lista
    return [kw.strip("' ") for kw in keywords_list]

# Aplicar la función a la columna de palabras clave
movies['keywords'] = movies['keywords'].apply(extract_keywords)

# Verificar el resultado
print(movies['keywords'].head())


0    [['differ, asid, put, learn, eventu, duo, owne...
1    [['creatur, terrifi, monkey, evil, rhinocero, ...
2    [['time, hot, cook, seafood, interest, le, awa...
3    [['way, better, determin, talk, robin, glo, be...
4    [['kid, grandchild, arriv, chang, like, home, ...
Name: keywords, dtype: object


In [36]:
movies['keywords'] = movies['keywords'].apply(lambda x: [item.strip() for sublist in x for item in sublist.replace('[', '').replace(']', '').split(',')])

# Verificar el resultado
print(movies['keywords'].head())

0    ['differ, asid, put, learn, eventu, duo, owner...
1    ['creatur, terrifi, monkey, evil, rhinocero, g...
2    ['time, hot, cook, seafood, interest, le, away...
3    ['way, better, determin, talk, robin, glo, ber...
4    ['kid, grandchild, arriv, chang, like, home, s...
Name: keywords, dtype: object


In [43]:
# Función para limpiar las palabras clave
def clean_keyword(keyword):
    return keyword.replace("'", "")

# Aplicar la función de limpieza a la columna
movies['keywords_cleaned'] = movies['keywords'].apply(lambda keywords: [clean_keyword(keyword) for keyword in keywords])

# Elimina el último elemento de cada lista de palabras clave
movies['keywords_cleaned'] = movies['keywords_cleaned'].apply(lambda keywords: keywords[:-1])

Vamos a imprimir por genero las distintas palabras mas frecuentes

In [48]:
from collections import Counter

# Obtener todos los géneros únicos
all_genres = set([genre for genres in movies['genero_names'] for genre in genres])

# Iterar sobre cada género y contar las palabras clave más comunes
for genre in all_genres:
    # Filtrar las películas de este género
    genre_movies = movies[movies['genero_names'].apply(lambda genres: genre in genres)]
    
    # Concatenar todas las palabras clave en una lista
    genre_keywords = [keyword for keywords in genre_movies['keywords_cleaned'] for keyword in keywords]
    
    # Contar las ocurrencias de cada palabra clave
    keyword_counts = Counter(genre_keywords)
    
    # Imprimir las 10 palabras clave más comunes para este género
    print(f"Las 10 palabras clave más comunes para el género {genre} son:")
    for keyword, count in keyword_counts.most_common(10):
        print(f"Keyword: {keyword}, Count: {count}")
    print("\n")


Las 10 palabras clave más comunes para el género a son:
Keyword: life, Count: 6240
Keyword: young, Count: 4259
Keyword: one, Count: 4187
Keyword: love, Count: 3976
Keyword: find, Count: 3925
Keyword: stori, Count: 3694
Keyword: film, Count: 3561
Keyword: new, Count: 3347
Keyword: man, Count: 3254
Keyword: woman, Count: 3153


Las 10 palabras clave más comunes para el género C son:
Keyword: life, Count: 2765
Keyword: get, Count: 2425
Keyword: find, Count: 2292
Keyword: one, Count: 2277
Keyword: new, Count: 1967
Keyword: two, Count: 1852
Keyword: love, Count: 1847
Keyword: friend, Count: 1812
Keyword: young, Count: 1778
Keyword: take, Count: 1755


Las 10 palabras clave más comunes para el género o son:
Keyword: life, Count: 5572
Keyword: one, Count: 4522
Keyword: find, Count: 4260
Keyword: young, Count: 3835
Keyword: new, Count: 3640
Keyword: love, Count: 3623
Keyword: get, Count: 3522
Keyword: film, Count: 3488
Keyword: take, Count: 3362
Keyword: two, Count: 3261


Las 10 palabras clav

In [55]:
movies

,budget,id_movie,original_language,overview,popularity,release_date,revenue,runtime,tagline,title,...,production_countries_names,production_countries_iso,spoken_languages_names,director,principal_actor,topic,processed_overview,keywords,keywords2,keywords_cleaned
0,30000000.0,862,en,led woody andy toy live happily room andy birt...,21.946943,1995-10-30,373554033.0,81.0,0,Toy Story,...,['United States of America'],['US'],['English'],['John Lasseter'],NaN,13,led woodi andi toy live happili room andi birt...,"['differ, asid, put, learn, eventu, duo, owner...",[diffe],"[differ, asid, put, learn, eventu, duo, owner,..."
1,65000000.0,8844,en,sibling judy peter discover enchanted board ga...,17.015539,1995-12-15,262797249.0,104.0,Roll the dice and unleash the excitement!,Jumanji,...,['United States of America'],['US'],"['English', 'Français']",['Joe Johnston'],Robin Williams,5,sibl judi peter discov enchant board game open...,"['creatur, terrifi, monkey, evil, rhinocero, g...",[creatu],"[creatur, terrifi, monkey, evil, rhinocero, gi..."
2,0.0,15602,en,family wedding reignites ancient feud nextdoor...,11.712900,1995-12-22,0.0,101.0,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,...,['United States of America'],['US'],['English'],['Howard Deutch'],NaN,9,famili wed reignit ancient feud nextdoor neigh...,"['time, hot, cook, seafood, interest, le, away...",[tim],"[time, hot, cook, seafood, interest, le, away,..."
3,16000000.0,31357,en,cheated mistreated stepped woman holding breat...,3.859495,1995-12-22,81452156.0,127.0,Friends are the people who let you be yourself...,Waiting to Exhale,...,['United States of America'],['US'],['English'],['Forest Whitaker'],Whitney Houston,13,cheat mistreat step woman hold breath wait elu...,"['way, better, determin, talk, robin, glo, ber...",[wa],"[way, better, determin, talk, robin, glo, bern..."
4,0.0,11862,en,george bank recovered daughter wedding receive...,8.387519,1995-02-10,76578911.0,106.0,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,...,['United States of America'],['US'],['English'],['Charles Shyer'],Steve Martin,13,georg bank recov daughter wed receiv news preg...,"['kid, grandchild, arriv, chang, like, home, s...",[ki],"[kid, grandchild, arriv, chang, like, home, se..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45447,0.0,30840,en,yet another version classic epic enough variat...,5.683753,1991-05-13,0.0,104.0,0,Robin Hood,...,"['Canada', 'Germany', 'United Kingdom', 'Unite...","['CA', 'DE', 'GB', 'US']",['English'],['John Irvin'],Patrick Bergin,2,yet anoth version classic epic enough variat m...,"['variat, darker, tone, photographi, marian, m...",[varia],"[variat, darker, tone, photographi, marian, ma..."
45448,0.0,111109,tl,artist struggle finish work storyline cult pla...,0.178241,2011-11-17,0.0,360.0,0,Century of Birthing,...,['Philippines'],['PH'],[''],['Lav Diaz'],NaN,2,artist struggl finish work storylin cult play ...,"['storylin, artist, cult, play, struggl, work,...",[storyli],"[storylin, artist, cult, play, struggl, work, ..."
45449,0.0,67758,en,one hit go wrong professional assassin end sui...,0.903007,2003-08-01,0.0,90.0,A deadly game of wits.,Betrayal,...,['United States of America'],['US'],['English'],['Mark L. Lester'],NaN,9,one hit go wrong profession assassin end suitc...,"['belong, suitcas, million, wrong, full, mob, ...",[belon],"[belong, suitcas, million, wrong, full, mob, p..."
45450,0.0,227506,en,small town live two brother one minister one h...,0.003503,1917-10-21,0.0,87.0,0,Satan Triumphant,...,['Russia'],['RU'],[],['Yakov Protazanov'],NaN,13,small town live two brother one minist one hun...,"['ehem, cuckold, earthli, stormi, viewpoint, a...",[ehe],"[ehem, cuckold, earthli, stormi, viewpoint, ad..."


In [60]:
from collections import defaultdict, Counter

# Crea un diccionario para almacenar las cuentas de palabras clave para cada género
keyword_counts_by_genre = defaultdict(Counter)

# Itera a través de las filas del marco de datos
for index, row in movies.iterrows():
    genero_names = ast.literal_eval(row['genero_names'])  # Convierte la cadena en una lista
    keywords_cleaned = row['keywords_cleaned']

    # Itera a través de los géneros en la lista de géneros
    for genero in genero_names:
        # Suma las cuentas de palabras clave para este género
        keyword_counts_by_genre[genero].update(keywords_cleaned)

# Imprime las 10 palabras clave más comunes para cada género
for genero, keyword_counts in keyword_counts_by_genre.items():
    print(f"Las 10 palabras clave más comunes para el género {genero} son:")
    for keyword, count in keyword_counts.most_common(10):
        print(f"Keyword: {keyword}, Count: {count}")

Las 10 palabras clave más comunes para el género Animation son:
Keyword: anim, Count: 310
Keyword: find, Count: 309
Keyword: world, Count: 289
Keyword: one, Count: 281
Keyword: life, Count: 273
Keyword: new, Count: 259
Keyword: friend, Count: 251
Keyword: take, Count: 236
Keyword: must, Count: 223
Keyword: young, Count: 209
Las 10 palabras clave más comunes para el género Comedy son:
Keyword: life, Count: 2249
Keyword: get, Count: 1997
Keyword: find, Count: 1820
Keyword: one, Count: 1774
Keyword: love, Count: 1646
Keyword: new, Count: 1634
Keyword: friend, Count: 1540
Keyword: two, Count: 1486
Keyword: take, Count: 1374
Keyword: young, Count: 1356
Las 10 palabras clave más comunes para el género Family son:
Keyword: find, Count: 509
Keyword: life, Count: 481
Keyword: famili, Count: 440
Keyword: friend, Count: 430
Keyword: new, Count: 424
Keyword: get, Count: 401
Keyword: young, Count: 361
Keyword: one, Count: 356
Keyword: take, Count: 344
Keyword: world, Count: 332
Las 10 palabras clav

La aparición frecuente de palabras como "find", "life", "young" y "friend" en múltiples géneros demuestra que existen temas universales que podrían ser esenciales en la conexión con las preferencias de los espectadores. Por lo tanto, estos datos pueden ser utilizados para mejorar la precisión de las recomendaciones, alineando las películas sugeridas con los intereses y gustos individuales de los usuarios.

Además, al explorar y clasificar estas palabras clave dentro de cada género, el sistema de recomendación puede ofrecer opciones más personalizadas y dirigidas, mejorando así la experiencia del usuario. Por ejemplo, si un usuario muestra una fuerte inclinación hacia películas que contienen el tema "amistad", el sistema podría recomendar más películas en diferentes géneros que también enfatizan este tema.

## Conclusion final y variables decisoras

Basándonos en nuestro análisis previo, se puede inferir que las variables que podrían ser más útiles para el modelo de recomendación son:

- Género de la película: Dado que cada género tiene su propio conjunto único de características y atrae a diferentes audiencias, es probable que el género sea un predictor valioso en un modelo de recomendación.

- Collection: Esta variable va a ser util cuando este disponible para identificar otras peliculas de la misma coleccion.

- Descripción de la película (Overview): A través de técnicas de procesamiento de lenguaje natural, la descripción de una película puede ser utilizada para encontrar similitudes en el contenido entre diferentes películas. Si una persona disfruta de una película, es probable que disfrute de películas con descripciones similares. Por lo tanto, este podría ser un predictor fuerte en un modelo de recomendación basado en contenido. Se utilizara mediante la coincidencia de Topicos.

En resumen, nuestro análisis nos ha proporcionado una visión detallada de los datos de las películas de TMDB. Hemos identificado algunas características interesantes y patrones en los datos, y hemos desarrollado un método para medir la similitud entre las descripciones de las películas. Este análisis puede ser un punto de partida útil para tareas futuras, como la construcción de un sistema de recomendación de películas o la realización de un análisis más profundo de las características de las películas.

In [63]:
df.to_csv('movies.csv')

In [6]:
pd.set_option('display.max_columns', None)

In [8]:
def recommend_movies(movie_title, movies, n_recommendations=5):
    # Extraer las características de la película seleccionada
    selected_movie = movies[movies['title'] == movie_title].iloc[0]

    # Calcular similitud basada en colección
    similarity = (movies['name_collection'] == selected_movie['name_collection']) * 10 if pd.notnull(selected_movie['name_collection']) else 0

    # Calcular similitud basada en tópico
    similarity += (movies['topic'] == selected_movie['topic']) * 0.5

    # Calcular similitud basada en género
    similarity += movies['genero_names'].apply(lambda x: sum(g in x for g in selected_movie['genero_names'])) * 0.2

    # Calcular similitud basada en palabras clave
    similarity += movies['keywords_cleaned'].apply(lambda x: sum(k in x for k in selected_movie['keywords_cleaned'])) * 0.5

    # Obtener las películas más similares
    top_n = movies.loc[similarity.nlargest(n_recommendations + 1).index[1:]]

    return top_n

recommend_movies('Jumanji', df, 5)


,budget,id_movie,original_language,overview,popularity,release_date,revenue,runtime,tagline,title,vote_average,vote_count,release_year,return,genero_ids,genero_names,id_collection,name_collection,production_companies_ids,production_companies_names,production_countries_names,production_countries_iso,spoken_languages_names,director,principal_actor,topic,processed_overview,keywords,keywords_cleaned
1939,0.0,10122,en,1986 disney adventure tell story 12yearold dav...,5.526865,1986-07-30,18564613.0,90.0,Come along on the greatest adventure of the su...,Flight of the Navigator,6.9,227.0,1986,0.0,"[10751, 878, 12]","['Family', 'Science Fiction', 'Adventure']",NaN,NaN,"[2, 1340, 11670]","['Walt Disney Pictures', 'New Star Entertainme...",['United States of America'],['US'],['English'],['Randal Kleiser'],Joey Cramer,6,1986 disney adventur tell stori 12yearold davi...,"[""'perplex"", 'overjoy', 'fla', 'lauderdal', 'u...","['perplex', 'overjoy', 'fla', 'lauderdal', 'uf..."
1949,0.0,34774,en,search castaway 1962 walt disney production fe...,2.679615,1962-12-01,0.0,98.0,"A Thousand Thrills, And Hayley Mills!",In Search of the Castaways,6.2,20.0,1962,0.0,"[28, 12, 18, 10751]","['Action', 'Adventure', 'Drama', 'Family']",NaN,NaN,[3166],['Walt Disney Productions'],['United States of America'],['US'],['English'],['Robert Stevenson'],NaN,2,search castaway 1962 walt disney product featu...,"[""'1868"", 'hawley', 'lowel', 'stevenson', 'che...","['1868', 'hawley', 'lowel', 'stevenson', 'chev..."
8131,0.0,19893,en,many kid might say best friend puppy pony 12ye...,2.439867,1963-08-14,0.0,87.0,The wonderful story of a boy and his amazing u...,Flipper,5.1,17.0,1963,0.0,"[12, 18, 10751]","['Adventure', 'Drama', 'Family']",NaN,NaN,[5328],['Ivan Tors Productions'],['United States of America'],['US'],['English'],['James B. Clark'],Chuck Connors,2,mani kid might say best friend puppi poni 12ye...,"[""'mitzi"", 'fin', 'dolphinpow', '196468', 'hal...","['mitzi', 'fin', 'dolphinpow', '196468', 'halp..."
42848,0.0,334939,de,grandfather car accident isabel 11 take search...,0.906563,2015-06-24,0.0,0.0,0,Sword of D'Artagnan,7.1,8.0,2015,0.0,"[10751, 12]","['Family', 'Adventure']",NaN,NaN,"[15163, 35206, 50837]","['Algemene Vereniging Radio Omroep (AVRO)', 'L...",['Netherlands'],['NL'],['Nederlands'],['Dennis Bots'],NaN,5,grandfath car accid isabel 11 take search swor...,"[""'artagnan"", 'illustri', 'rik', 'persever', '...","['artagnan', 'illustri', 'rik', 'persever', 'b..."
12613,0.0,14784,en,hospital outskirt 1920s los angeles injured st...,15.855108,2008-01-03,3669465.0,117.0,A little blessing in disguise.,The Fall,7.7,430.0,2008,0.0,"[12, 14, 18]","['Adventure', 'Fantasy', 'Drama']",NaN,NaN,"[911, 3586, 11150, 11151, 11152, 11153]","['Roadside Attractions', 'Absolute Entertainme...","['United States of America', 'India']","['US', 'IN']","['English', 'Română', 'Latin']",['Tarsem Singh'],NaN,5,hospit outskirt 1920 lo angel injur stuntman b...,"[""'fractur"", 'stuntman', 'blur', 'fantast', 'o...","['fractur', 'stuntman', 'blur', 'fantast', 'ou..."


In [ ]:
df[df['title'] == 'Jurassic Park']

In [ ]:
# Obtener las palabras clave para cada película
keywords = []
for i in range(tfidf_matrix.shape[0]):
    tfidf_scores = tfidf_matrix[i]
    words = vectorizer.inverse_transform(tfidf_scores)
    keywords.append(words)

movies['keywords'] = keywords